# 4.2 - Sistemas de recomendación


#### Filtro Colaborativo

Técnica utilizada por algunos sistemas recomendadores, los cuales  suelen incluir conjuntos de datos muy grandes. Hay dos tipos de filtros colaborativos:

+ Basados en Usuarios (User-based):

Se basan en una premisa simple de similitud entre gustos, es decir, en que si una persona A tiene la misma opinión que una persona B sobre un tema, A es más probable que tenga la misma opinión que B en otro tema diferente que la opinión que tendría una persona elegida azar. 

Tiene ciertos inconvenientes. 
Requiere evaluaciones previas, sin algunas evaluaciones de usuarios, no se tienen pistas sobre las evaluaciones de los nuevos usuarios.
No es una matriz densa, usualmente los usuarios dan su rating sobre algunos items y no sobre toda la base de datos, lo que puede dificultar algunos casos por falta de datos pero puede ayudar ya que no se tiene que hacer los cálculos para todas las combinaciones posibles.
Altos costos si los perfiles de usuarios cambian, si los gustos cambian o hay ruido en los datos (ejemplo, 2 personas usan la misma cuenta en el sistema) todo el modelo del sistema debe ser recalculado.

+ Basados en Items (Items-based):

Buscan la similitud entre elementos, utilizan distribuciones de calificación por artículo, no por usuario, lo que puede ayudar a sobrellevar algunos de los problemas de los modelos User-based. Con más usuarios que elementos, cada elemento tiende a tener más calificaciones que cada usuario, por lo que la calificación promedio de un elemento generalmente no cambia rápidamente. Esto conduce a distribuciones de calificación más estables en el modelo, por lo que no es necesario reconstruir el modelo con tanta frecuencia. Cuando los usuarios consumen y luego califican un artículo, los elementos similares de ese elemento se seleccionan del modelo de sistema existente y se agregan a las recomendaciones del usuario.

### Ejemplo Peliculas (User-based)

In [ ]:
def plot(m1, m2):
    
    x=ratings.T[m1]  # ratings de las peliculas
    y=ratings.T[m2]
    
    n=list(ratings.T.index)  # nombres de los usuarios
    
    plt.figure(figsize=(10, 5))
    
    plt.scatter(x, y, s=0) # scatter vacio...para luego poner los nombres
    
    plt.title('Espacio para {} VS. {}'.format(m1, m2), fontsize=14)
    plt.xlabel(m1, fontsize=14)
    plt.ylabel(m2, fontsize=14)
    
    for i,e in enumerate(n):
        plt.annotate(e, (x[i], y[i]), fontsize=12)  # se escribe el nombre en las posiciones correspondientes
    plt.show();

##### metricas de distancia

https://docs.scipy.org/doc/scipy/reference/generated/scipy.spatial.distance.pdist.html

https://docs.scipy.org/doc/scipy-0.14.0/reference/generated/scipy.spatial.distance.squareform.html

https://docs.scipy.org/doc/scipy-0.14.0/reference/spatial.distance.html

**euclidea**

$$d=\sqrt{x^2 + y^2}$$

$$d=\sqrt{x_1^2 + x_2^2 + \ldots + x_n^2}$$

**cityblock - manhattan**

$$d=x + y$$

**similitud del coseno**

In [ ]:
import scipy.spatial.distance as distance
import numpy as np
import math

**producto escalar**

$$(1, 2, 3)·(2, 1, 1)= 1·2 + 2·1 + 3·1 =7$$

In [ ]:
def dot(v1, v2):
    return sum(map(lambda x: x[0]*x[1], zip(v1,v2)))

$$\cos{\theta}=\frac{v_{1}·v_{2}}{|v_{1}|·|v_{2}|}$$

donde:

$|V_{1}|=\sqrt{x_1^2+x_2^2+x_3^2}$

In [ ]:
def similitud_coseno(v1, v2):
    
    prod=dot(v1, v2)
    
    len1=math.sqrt(dot(v1, v1))
    len2=math.sqrt(dot(v2, v2))
    
    return prod/(len1*len2)

In [ ]:
def plot_v(v):
    origen=(0,0)

    plt.quiver(*origen, v[0,0], v[0,1], color=['r'], scale=60)
    plt.quiver(*origen, v[1,0], v[1,1], color=['b'], scale=60)
    
    a=v[0]
    b=v[1]
    
    sim_cos=similitud_coseno(a, b)
    sim_euc=distance.euclidean(a, b)
    
    tit='Coseno: {}, Euclidea: {}'.format(sim_cos.round(2), sim_euc.round(2))
    plt.title(tit)
    plt.grid()
    plt.legend(['A', 'B']);

In [ ]:
plt.figure(figsize=(20,20))

# caso 1
v=np.array([[0,10], [0, 20]])
plt.subplot(2,2,1)
plot_v(v)

# caso 2
v=np.array([[10,10], [0, 20]])
plt.subplot(2,2,2)
plot_v(v)

# caso 3
v=np.array([[10,0], [0, 10]])
plt.subplot(2,2,3)
plot_v(v)

# caso 4
v=np.array([[0,5], [0, -5]])
plt.subplot(2,2,4)
plot_v(v)

### Generar recomendaciones

In [ ]:
tom={'Aquaman': 2, 
     'Avengers: Infinity War':1, 
     'Black Panther':5,
     'Bohemian Rhapsody':5, 
     'Deadpool 2':2,
     'Fantastic Beasts: The Crimes of Grindelwald':3, 
     'Incredibles 2':3,
     'Jurassic World: Fallen Kingdom':4, 
     'Mission: Impossible – Fallout':3,
     'Venom':3}

#### Score de similitud